# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [ ]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
from pymongo import UpdateOne

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(host='localhost',port=27017)

In [ ]:
# confirm that our new database was created
print(mongo.list_database_names())

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our new database
print(db.list_collection_names())

In [ ]:
# review a document in the establishments collection
print(db.establishments.find_one())

In [ ]:
# assign the collection to a variable
establishments = db['establishments']
establishments

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [ ]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    '_id': 1,  # MongoDB will automatically generate this if omitted
    'FHRSID': None,
    'ChangesByServerID': None,
    'LocalAuthorityBusinessID': None,
    'BusinessName': 'Penang Flavours',
    'BusinessType': None,
    'BusinessTypeID': None,
    'AddressLine1': None,
    'AddressLine2': None,
    'AddressLine3': None,
    'AddressLine4': None,
    'PostCode': None,
    'Phone': None,
    'RatingValue': None,
    'RatingKey': None,
    'RatingDate': None,
    'LocalAuthorityCode': None,
    'LocalAuthorityName': None,
    'LocalAuthorityWebSite': None,
    'LocalAuthorityEmailAddress': None,
    'scores': {
        'Hygiene': None,
        'Structural': None,
        'ConfidenceInManagement': None
    },
    'SchemeType': None,
    'geocode': {
        'longitude': None,
        'latitude': None
    },
    'RightToReply': None,
    'Distance': None,
    'NewRatingPending': None,
    'meta': {
        'dataSource': None,
        'extractDate': None,
        'itemCount': None,
        'returncode': None,
        'totalCount': None,
        'totalPages': None,
        'pageSize': None,
        'pageNumber': None
    },
    'links': [
        {
            'rel': None,
            'href': None
        }
    ]
}

In [ ]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

In [ ]:
# Check that the new restaurant was inserted
pprint(establishments.find_one({'BusinessName':'Penang Flavours'}))

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [ ]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
establishments.find_one(
        {'BusinessType': 'Restaurant/Cafe/Canteen'},
    {'BusinessTypeID': 1, 'BusinessType': 1, '_id': 0}
)

3. Update the new restaurant with the `BusinessTypeID` you found.

In [ ]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one(
    {'BusinessName': 'Penang Flavours'},  # Filter to find the specific document
    {'$set': {'BusinessTypeID': 1}}      # Update the BusinessTypeID field to 1
)

In [ ]:
# Confirm that the new restaurant was updated
establishments.find_one(
        {'BusinessName': 'Penang Flavours'},
)

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [ ]:
# Find how many documents have LocalAuthorityName as "Dover"
count = establishments.count_documents({'LocalAuthorityName': 'Dover'})

count

In [ ]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many({'LocalAuthorityName': 'Dover'})

In [ ]:
# Check if any remaining documents include Dover
count = establishments.count_documents({'LocalAuthorityName': 'Dover'})

count

In [ ]:
# Check that other documents remain with 'find_one'
establishments.find_one({'LocalAuthorityName': 'Dover'})

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [ ]:
# Change the data type from String to Decimal for longitude and latitude

# Create a list to store update operations
updates = []

# Iterate over all documents
for doc in establishments.find({'geocode.longitude': {'$type': 'string'}, 'geocode.latitude': {'$type': 'string'}}):
    # Convert longitude and latitude from string to float (for double type in MongoDB)
    longitude = float(doc['geocode']['longitude'])
    latitude = float(doc['geocode']['latitude'])
    
    # Prepare the update operation
    updates.append(
        UpdateOne(
            {'_id': doc['_id']},  # Filter by _id to target the specific document
            {'$set': {'geocode.longitude': longitude, 'geocode.latitude': latitude}}
        )
    )

# Execute the batch update
if updates:
    result = establishments.bulk_write(updates)
    print("Documents updated:", result.modified_count)
else:
    print("No documents found with longitude and latitude as strings.")

Use `update_many` to convert `RatingValue` to integer numbers.

In [ ]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [ ]:
# Change the data type from String to Integer for RatingValue

result = establishments.update_many(
    {'RatingValue': {'$regex': '^\d+$'}},  # Match documents where RatingValue is a numeric string
    [{'$set': {'RatingValue': {'$toInt': '$RatingValue'}}}]  # Convert RatingValue to integer
)

# Print the number of documents modified
print("Documents updated:", result.modified_count)

In [ ]:
# Check that the coordinates and rating value are now numbers

# Check if there are documents where 'longitude', 'latitude', and 'RatingValue' are numeric
longitude_numeric_count = establishments.count_documents({'geocode.longitude': {'$type': 'double'}})
latitude_numeric_count = establishments.count_documents({'geocode.latitude': {'$type': 'double'}})
rating_value_numeric_count = establishments.count_documents({'RatingValue': {'$type': 'int'}})

# Print results
print("Number of documents with 'longitude' as float:", longitude_numeric_count)
print("Number of documents with 'latitude' as float:", latitude_numeric_count)
print("Number of documents with 'RatingValue' as integer:", rating_value_numeric_count)